In [ ]:
#### CSE 587: Project 3
#### Members: Thankam Abish, Krishna Menon, Lucas Bietzel

In [1]:
import findspark

In [2]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
sc = pyspark.SparkContext()

In [5]:
from pyspark.sql import *

In [6]:
spark = SparkSession \
    .builder \
    .appName("Pyspark demo") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [7]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [8]:
###ADJUST TO CORRECT FILE PATHS

dfs = spark.read.csv('/home/cse587/A3Data/train.csv', escape='"', inferSchema = 'True',header="True")
testdfs = spark.read.csv('/home/cse587/A3Data/test.csv', escape='"', inferSchema = 'True',header="True")

In [9]:
dfs.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [10]:
from pyspark.ml.feature import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

### SMALL DATAFRAME USED FOR TESTING
#dfs = spark.createDataFrame([
 #   (12340, "Hi hi HI HI  , HI HI", "['Drama','Comedy']"),
  #  (123545, "I wish Java could use case classes", "['Drama','Comedy','Action']"),
   # (2123, "things are really neat. wow", "['Action','Comedy','Horror']"),
    #(21523, "fat cats are cute", "['Action','Comedy','Horror']")], 
    #["id", "plot", "genre"])
    
#testdfs = spark.createDataFrame([
 #   (12640, "Hi wish cats")],
  #  ["id", "plot"])

In [11]:
#### FIND BAD DATA

def to_fix(x):
    try:
        eval(x)
        return True
    except:
        return False


fix_udf = udf(lambda x: to_fix(x), BooleanType())
dfs2 = dfs.withColumn('validity', fix_udf('genre'))
dfs2.show()



+--------+--------------------+--------------------+--------------------+--------+
|movie_id|          movie_name|                plot|               genre|validity|
+--------+--------------------+--------------------+--------------------+--------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|    true|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|    true|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|    true|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|    true|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|    true|
| 5272176|            End Game|The president is ...|['Action/Adventur...|    true|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|    true|
|24225279|                Sing|The story begins ...|           ['Drama']|    true|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|    true|
|205

In [12]:
dfs = dfs2.filter(col("validity")==True)
dfs.count()

31106

In [13]:
dfs = dfs.dropna()
dfs.count()

31106

In [14]:
#plot text cleaning, shows the clean text 
def clean(x):
    dfs2 = x.withColumn('plot', (col('plot')).cast(StringType()))
    text = regexp_replace(dfs2['plot'], "[^a-zA-Z ]","")
    text = regexp_replace(text, ' +', ' ')
    dfs2 = dfs2.withColumn('plot', text)
    dfs2.show()
    dfs2 = dfs2.withColumn('plot', lower(col('plot')))
    dfs2.show()
    c = split(dfs2['plot'],' ')
    dfs2 = dfs2.withColumn('plot', c)
    dfs2.show()

    ##stop word removal
    remover = StopWordsRemover(inputCol="plot", outputCol="filtered")
    dfs2 = remover.transform(dfs2)
    dfs2 = dfs2.withColumn('plot', col('filtered'))
    dfs2 = dfs2.drop('filtered')
    return dfs2

dfs2 = clean(dfs2)
testdfs = clean(testdfs)

+--------+--------------------+--------------------+--------------------+--------+
|movie_id|          movie_name|                plot|               genre|validity|
+--------+--------------------+--------------------+--------------------+--------+
|23890098|          Taxi Blues|Shlykov a hardwor...|['World cinema', ...|    true|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|    true|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|    true|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|    true|
|  595909|   A Cry in the Dark|Seventhday Advent...|['Crime Fiction',...|    true|
| 5272176|            End Game|The president is ...|['Action/Adventur...|    true|
| 1952976|          Dark Water|plot The film ope...|['Thriller', 'Dra...|    true|
|24225279|                Sing|The story begins ...|           ['Drama']|    true|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|    true|
|205

+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|
+--------+--------------------+--------------------+
| 1335380|              Exodus|[the, film, is, b...|
|29062594|A la salida nos v...|[a, group, of, te...|
| 9252321|   Come Back, Africa|[this, story, of,...|
|13455076|       A Merry Mixup|[the, stooges, pl...|
|24165951|        Getting Even|[a, soldieroffort...|
| 1925869|  River of No Return|[set, in, the, no...|
|10799612|          Amici miei|[like, in, many, ...|
|28238240|Mickey's Big Game...|[mickey, and, the...|
|17124781|The Good, the Bad...|[in, the, desert,...|
|28207941|    The Dancing Fool|[bimbo, and, koko...|
|19174305|              Tahaan|[tahaan, lives, w...|
|18392317|     Mysterious Mose|[betty, is, start...|
|34420857|Kelviyum Naane Pa...|[nirmal, karthik,...|
| 4039635|   First on the Moon|[a, group, of, jo...|
| 8034072|  Journey of a Woman|[vaibhavari, saha...|
| 4016437|     Sophie's Choice|[in, the, movie

In [ ]:
dfs2.printSchema()
dfs2.show()
testdfs.printSchema()
dfs2.show()

In [ ]:
####################################################################
############     PART 1   (ONLY RUN ONE PART)
####################################################################
cv = CountVectorizer(inputCol="plot", outputCol="features")
model = cv.fit(dfs2)
dfs3 = model.transform(dfs2)
testdfs3 = model.transform(testdfs)

In [15]:
####################################################################
############     PART 2    (ONLY RUN ONE PART)
####################################################################
cv = CountVectorizer(inputCol="plot", outputCol="rawFeatures")
model = cv.fit(dfs2)
dfs3 = model.transform(dfs2)
testdfs3 = model.transform(testdfs)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(dfs3)
dfs3 = idfModel.transform(dfs3)
testdfs3 = idfModel.transform(testdfs3)

In [ ]:
####################################################################
############     PART 3    (ONLY RUN ONE PART)
####################################################################
word2Vec = Word2Vec(vectorSize = 100, inputCol="plot", outputCol="features")
w2vmodel = word2Vec.fit(dfs2)
dfs3 = w2vmodel.transform(dfs2)
testdfs3 = w2vmodel.transform(testdfs)

In [16]:
dfs3.printSchema()
dfs3.count()
testdfs3.printSchema()
dfs3.show()
testdfs3.show()

root
 |-- movie_id: string (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genre: string (nullable = true)
 |-- validity: boolean (nullable = true)
 |-- rawFeatures: vector (nullable = true)
 |-- features: vector (nullable = true)

root
 |-- movie_id: integer (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rawFeatures: vector (nullable = true)
 |-- features: vector (nullable = true)

+--------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|validity|         rawFeatures|            features|
+--------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+
|23890098|          Taxi Blues|[shlyko

In [17]:
#### DETECT BAD DATA

def to_fix(x): 
    try:
        eval(x)
        return True
    except:
        return False

fix_udf = udf(lambda x: to_fix(x), BooleanType())
dfs4 = dfs3.withColumn('validity', fix_udf('genre'))
dfs4.show()

dfs3 = dfs4.filter(col("validity")==True)

+--------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|validity|         rawFeatures|            features|
+--------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+
|23890098|          Taxi Blues|[shlykov, hardwor...|['World cinema', ...|    true|(154617,[125,179,...|(154617,[125,179,...|
|31186339|    The Hunger Games|[nation, panem, c...|['Action/Adventur...|    true|(154617,[2,4,7,11...|(154617,[2,4,7,11...|
|20663735|          Narasimham|[poovalli, induch...|['Musical', 'Acti...|    true|(154617,[1,5,9,11...|(154617,[1,5,9,11...|
| 2231378|  The Lemon Drop Kid|[lemon, drop, kid...|          ['Comedy']|    true|(154617,[7,8,12,1...|(154617,[7,8,12,1...|
|  595909|   A Cry in the Dark|[seventhday, adve...|['Crime Fiction',...|    true|(154617,[2,8,9,14...|(154617,[2,8,9,14...|


In [18]:
#### CREATE BINARY LABEL COLUMNS FOR GENRE

genres = ['Drama','Comedy','Romance Film','Thriller','Action',
          'World cinema','Crime Fiction','Horror','Black-and-white',
          'Indie','Action/Adventure','Adventure','Family Film',
          'Short Film','Romantic drama','Animation','Musical',
          'Science Fiction','Mystery','Romantic comedy']

def to_genre(x):
    a = eval(x)
    b = []
    for i in genres:
        if i in a:
            b.append(1)
        else:
            b.append(0)
    return b

to_list_udf = udf(lambda x: to_genre(x), ArrayType(IntegerType()))
dfs6 = dfs3.withColumn('genre', to_list_udf('genre'))
dfs6.show()

+--------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|validity|         rawFeatures|            features|
+--------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+
|23890098|          Taxi Blues|[shlykov, hardwor...|[1, 0, 0, 0, 0, 1...|    true|(154617,[125,179,...|(154617,[125,179,...|
|31186339|    The Hunger Games|[nation, panem, c...|[1, 0, 0, 0, 1, 0...|    true|(154617,[2,4,7,11...|(154617,[2,4,7,11...|
|20663735|          Narasimham|[poovalli, induch...|[1, 0, 0, 0, 1, 0...|    true|(154617,[1,5,9,11...|(154617,[1,5,9,11...|
| 2231378|  The Lemon Drop Kid|[lemon, drop, kid...|[0, 1, 0, 0, 0, 0...|    true|(154617,[7,8,12,1...|(154617,[7,8,12,1...|
|  595909|   A Cry in the Dark|[seventhday, adve...|[1, 0, 0, 0, 0, 1...|    true|(154617,[2,8,9,14...|(154617,[2,8,9,14...|


In [19]:
#### SEPARATED GENRE COLUMNS

for i in range(len(genres)):
    dfs6=dfs6.withColumn(genres[i],col('genre').getItem(i))
dfs6 = dfs6.drop('genre')
dfs6.show()

+--------+--------------------+--------------------+--------+--------------------+--------------------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+---------------+-------+---------------+
|movie_id|          movie_name|                plot|validity|         rawFeatures|            features|Drama|Comedy|Romance Film|Thriller|Action|World cinema|Crime Fiction|Horror|Black-and-white|Indie|Action/Adventure|Adventure|Family Film|Short Film|Romantic drama|Animation|Musical|Science Fiction|Mystery|Romantic comedy|
+--------+--------------------+--------------------+--------+--------------------+--------------------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+---------------+-------+---------------+
|23890098|          Taxi 

In [ ]:
dfs6.printSchema()
dfs6.show()

In [ ]:
# SINGLE MODEL CREATION
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'Drama', maxIter=100)
lrModel = lr.fit(dfs6)

In [ ]:
# THE TESTING FOR SINGLE MODEL CREATION
prediction = lrModel.transform(testdfs3)
prediction.show()

In [20]:
# ALL 20 BINARY CLASSIFIERS
import numpy as np
from pyspark.ml.classification import LogisticRegression
biglr = np.array([])
biglrModel = np.array([])
for i in range(len(genres)):
    biglr = np.append(biglr,LogisticRegression(featuresCol = 'features', labelCol = genres[i], maxIter=100))
    biglrModel = np.append(biglrModel, biglr[i].fit(dfs6))

In [21]:
# PREDICTIONS FROM BINARY CLASSIFIERS
preds = np.array([])
for i in range(len(genres)):
    a = biglrModel[i].transform(testdfs3)
    a = a.withColumn('prediction', col('prediction').cast(IntegerType()))
    preds = np.append(preds, a)
    preds[i].show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|          movie_name|                plot|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1335380|              Exodus|[film, based, eve...|(154617,[0,3,5,7,...|(154617,[0,3,5,7,...|[-840.80711264288...|           [0.0,1.0]|         1|
|29062594|A la salida nos v...|[group, teenagers...|(154617,[7,9,58,7...|(154617,[7,9,58,7...|[37.3441311695093...|[1.0,6.0485308714...|         0|
| 9252321|   Come Back, Africa|[story, zulu, fam...|(154617,[6,7,10,1...|(154617,[6,7,10,1...|[-30.853261959510...|[3.98656254646754...|         1|
|13455076|       A Merry Mixup|[stooges, play, t...|(154617,[0,68,77,...|(154617,[0,68,77,...|[114.050109036280.

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|          movie_name|                plot|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1335380|              Exodus|[film, based, eve...|(154617,[0,3,5,7,...|(154617,[0,3,5,7,...|[224.183312797815...|[1.0,4.3493495953...|         0|
|29062594|A la salida nos v...|[group, teenagers...|(154617,[7,9,58,7...|(154617,[7,9,58,7...|[35.0682345093492...|[0.99999999999999...|         0|
| 9252321|   Come Back, Africa|[story, zulu, fam...|(154617,[6,7,10,1...|(154617,[6,7,10,1...|[41.1960815014858...|[1.0,1.2846035796...|         0|
|13455076|       A Merry Mixup|[stooges, play, t...|(154617,[0,68,77,...|(154617,[0,68,77,...|[56.8379766011380.

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|          movie_name|                plot|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1335380|              Exodus|[film, based, eve...|(154617,[0,3,5,7,...|(154617,[0,3,5,7,...|[940.184336711015...|           [1.0,0.0]|         0|
|29062594|A la salida nos v...|[group, teenagers...|(154617,[7,9,58,7...|(154617,[7,9,58,7...|[205.332506806239...|[1.0,6.6869078547...|         0|
| 9252321|   Come Back, Africa|[story, zulu, fam...|(154617,[6,7,10,1...|(154617,[6,7,10,1...|[951.391365693245...|           [1.0,0.0]|         0|
|13455076|       A Merry Mixup|[stooges, play, t...|(154617,[0,68,77,...|(154617,[0,68,77,...|[291.654658731241.

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|          movie_name|                plot|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1335380|              Exodus|[film, based, eve...|(154617,[0,3,5,7,...|(154617,[0,3,5,7,...|[461.680741636991...|[1.0,3.1232122978...|         0|
|29062594|A la salida nos v...|[group, teenagers...|(154617,[7,9,58,7...|(154617,[7,9,58,7...|[46.6666501112063...|[1.0,5.4066887348...|         0|
| 9252321|   Come Back, Africa|[story, zulu, fam...|(154617,[6,7,10,1...|(154617,[6,7,10,1...|[157.563610941272...|[1.0,3.7238589222...|         0|
|13455076|       A Merry Mixup|[stooges, play, t...|(154617,[0,68,77,...|(154617,[0,68,77,...|[60.9883066594171.

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|          movie_name|                plot|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1335380|              Exodus|[film, based, eve...|(154617,[0,3,5,7,...|(154617,[0,3,5,7,...|[212.014589062395...|[1.0,8.3798045099...|         0|
|29062594|A la salida nos v...|[group, teenagers...|(154617,[7,9,58,7...|(154617,[7,9,58,7...|[30.2212726745939...|[0.99999999999992...|         0|
| 9252321|   Come Back, Africa|[story, zulu, fam...|(154617,[6,7,10,1...|(154617,[6,7,10,1...|[105.998877749880...|[1.0,9.2315006645...|         0|
|13455076|       A Merry Mixup|[stooges, play, t...|(154617,[0,68,77,...|(154617,[0,68,77,...|[63.5323852368835.

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|          movie_name|                plot|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1335380|              Exodus|[film, based, eve...|(154617,[0,3,5,7,...|(154617,[0,3,5,7,...|[288.506722819037...|[1.0,5.0480341900...|         0|
|29062594|A la salida nos v...|[group, teenagers...|(154617,[7,9,58,7...|(154617,[7,9,58,7...|[41.8172769692969...|[1.0,6.9021972130...|         0|
| 9252321|   Come Back, Africa|[story, zulu, fam...|(154617,[6,7,10,1...|(154617,[6,7,10,1...|[218.727587674386...|[1.0,1.0181590667...|         0|
|13455076|       A Merry Mixup|[stooges, play, t...|(154617,[0,68,77,...|(154617,[0,68,77,...|[88.1724470061125.

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|          movie_name|                plot|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1335380|              Exodus|[film, based, eve...|(154617,[0,3,5,7,...|(154617,[0,3,5,7,...|[203.767606009437...|[1.0,3.1978134105...|         0|
|29062594|A la salida nos v...|[group, teenagers...|(154617,[7,9,58,7...|(154617,[7,9,58,7...|[18.7637777301138...|[0.99999999290430...|         0|
| 9252321|   Come Back, Africa|[story, zulu, fam...|(154617,[6,7,10,1...|(154617,[6,7,10,1...|[134.856840213753...|[1.0,2.7065649179...|         0|
|13455076|       A Merry Mixup|[stooges, play, t...|(154617,[0,68,77,...|(154617,[0,68,77,...|[45.5054565036230.

In [23]:
### MERGE PREDICTIONS
final = preds[0].withColumn('id',col('movie_id').cast(StringType())).select(col('id'))
final.show()

for i in range(len(genres)):
    
    final = final.join(preds[i], final.id == preds[i].movie_id)
    final = final.withColumn(genres[i],col('prediction'))
    final = final.drop('movie_name','movie_id','plot','features','rawPrediction','probability','prediction','rawFeatures')
    final.show()

df = final.withColumn('prediction', 
                    concat(col('Drama')
                          ,lit(' '),col('Comedy')
                          ,lit(' '),col('Romance Film')
                          ,lit(' '),col('Thriller')
                          ,lit(' '),col('Action')
                          ,lit(' '),col('World cinema')
                          ,lit(' '),col('Crime Fiction')
                          ,lit(' '),col('Horror')
                          ,lit(' '),col('Black-and-white')
                          ,lit(' '),col('Indie')
                          ,lit(' '),col('Action/Adventure')
                          ,lit(' '),col('Adventure')
                          ,lit(' '),col('Family Film')
                          ,lit(' '),col('Short Film')
                          ,lit(' '),col('Romantic drama')
                          ,lit(' '),col('Animation')
                          ,lit(' '),col('Musical')
                          ,lit(' '),col('Science Fiction')
                          ,lit(' '),col('Mystery')
                          ,lit(' '),col('Romantic comedy')))
df.show()

+--------+
|      id|
+--------+
| 1335380|
|29062594|
| 9252321|
|13455076|
|24165951|
| 1925869|
|10799612|
|28238240|
|17124781|
|28207941|
|19174305|
|18392317|
|34420857|
| 4039635|
| 8034072|
| 4016437|
| 1520023|
|24589422|
|35068740|
|21132951|
+--------+
only showing top 20 rows

+--------+-----+
|      id|Drama|
+--------+-----+
| 1335380|    1|
|29062594|    0|
| 9252321|    1|
|13455076|    0|
|24165951|    0|
| 1925869|    1|
|10799612|    1|
|28238240|    0|
|17124781|    0|
|28207941|    0|
|19174305|    1|
|18392317|    0|
|34420857|    0|
| 4039635|    0|
| 8034072|    1|
| 4016437|    1|
| 1520023|    1|
|24589422|    1|
|35068740|    0|
|21132951|    0|
+--------+-----+
only showing top 20 rows

+--------+-----+------+
|      id|Drama|Comedy|
+--------+-----+------+
| 1335380|    1|     0|
|29062594|    0|     0|
| 9252321|    1|     0|
|13455076|    0|     1|
|24165951|    0|     0|
| 1925869|    1|     0|
|10799612|    1|     1|
|28238240|    0|     1|
|17124781|  

+--------+-----+------+------------+--------+------+------------+-------------+------+
|      id|Drama|Comedy|Romance Film|Thriller|Action|World cinema|Crime Fiction|Horror|
+--------+-----+------+------------+--------+------+------------+-------------+------+
| 1335380|    1|     0|           0|       0|     0|           0|            0|     0|
|29062594|    0|     0|           0|       0|     0|           0|            0|     0|
| 9252321|    1|     0|           0|       0|     0|           0|            0|     0|
|13455076|    0|     1|           0|       0|     0|           0|            0|     0|
|24165951|    0|     0|           0|       0|     1|           0|            0|     0|
| 1925869|    1|     0|           0|       0|     0|           0|            0|     0|
|10799612|    1|     1|           0|       0|     0|           0|            0|     0|
|28238240|    0|     1|           0|       0|     0|           0|            0|     0|
|17124781|    0|     0|           0|       

+--------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+
|      id|Drama|Comedy|Romance Film|Thriller|Action|World cinema|Crime Fiction|Horror|Black-and-white|Indie|Action/Adventure|Adventure|
+--------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+
| 1335380|    1|     0|           0|       0|     0|           0|            0|     0|              0|    0|               1|        0|
|29062594|    0|     0|           0|       0|     0|           0|            0|     0|              0|    0|               0|        0|
| 9252321|    1|     0|           0|       0|     0|           0|            0|     0|              0|    0|               0|        0|
|13455076|    0|     1|           0|       0|     0|           0|            0|     0|              0|    0|               0|        0|
|24165951|    0|     0|           0|       0|   

+--------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+
|      id|Drama|Comedy|Romance Film|Thriller|Action|World cinema|Crime Fiction|Horror|Black-and-white|Indie|Action/Adventure|Adventure|Family Film|Short Film|Romantic drama|
+--------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+
| 1335380|    1|     0|           0|       0|     0|           0|            0|     0|              0|    0|               1|        0|          0|         0|             0|
|29062594|    0|     0|           0|       0|     0|           0|            0|     0|              0|    0|               0|        0|          0|         0|             0|
| 9252321|    1|     0|           0|       0|     0|           0|            0|     0|              0|    0|               0|     

+--------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+
|      id|Drama|Comedy|Romance Film|Thriller|Action|World cinema|Crime Fiction|Horror|Black-and-white|Indie|Action/Adventure|Adventure|Family Film|Short Film|Romantic drama|Animation|Musical|
+--------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+
| 1335380|    1|     0|           0|       0|     0|           0|            0|     0|              0|    0|               1|        0|          0|         0|             0|        0|      0|
|29062594|    0|     0|           0|       0|     0|           0|            0|     0|              0|    0|               0|        0|          0|         0|             0|        0|      0|
| 9252321|    1|     0|           0|    

+--------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+---------------+-------+
|      id|Drama|Comedy|Romance Film|Thriller|Action|World cinema|Crime Fiction|Horror|Black-and-white|Indie|Action/Adventure|Adventure|Family Film|Short Film|Romantic drama|Animation|Musical|Science Fiction|Mystery|
+--------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+---------------+-------+
| 1335380|    1|     0|           0|       0|     0|           0|            0|     0|              0|    0|               1|        0|          0|         0|             0|        0|      0|              0|      0|
|29062594|    0|     0|           0|       0|     0|           0|            0|     0|              0|    0|               0|        0| 

+--------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+---------------+-------+---------------+--------------------+
|      id|Drama|Comedy|Romance Film|Thriller|Action|World cinema|Crime Fiction|Horror|Black-and-white|Indie|Action/Adventure|Adventure|Family Film|Short Film|Romantic drama|Animation|Musical|Science Fiction|Mystery|Romantic comedy|          prediction|
+--------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+---------------+-------+---------------+--------------------+
| 1335380|    1|     0|           0|       0|     0|           0|            0|     0|              0|    0|               1|        0|          0|         0|             0|        0|      0|              0|      0|              0|1 0 0 0 0 

In [24]:
### PREPARING FORMAT FOR CSV FILE

convertToCSV = df.select(col('id').alias('movie_id'),col('prediction'))
convertToCSV.show()

+--------+--------------------+
|movie_id|          prediction|
+--------+--------------------+
| 1335380|1 0 0 0 0 0 0 0 0...|
|29062594|0 0 0 0 0 0 0 0 0...|
| 9252321|1 0 0 0 0 0 0 0 0...|
|13455076|0 1 0 0 0 0 0 0 0...|
|24165951|0 0 0 0 1 0 0 0 0...|
| 1925869|1 0 0 0 0 0 0 0 0...|
|10799612|1 1 0 0 0 0 0 0 0...|
|28238240|0 1 0 0 0 0 0 0 0...|
|17124781|0 0 0 0 0 0 0 0 0...|
|28207941|0 0 0 0 0 0 0 1 1...|
|19174305|1 0 1 0 0 0 0 0 0...|
|18392317|0 0 0 0 0 0 0 0 0...|
|34420857|0 0 0 0 0 1 0 0 0...|
| 4039635|0 0 0 0 0 0 0 0 0...|
| 8034072|1 0 1 0 0 1 0 0 0...|
| 4016437|1 0 0 0 0 0 0 0 0...|
| 1520023|1 0 0 0 0 1 0 0 0...|
|24589422|1 0 1 0 0 0 0 0 0...|
|35068740|0 0 1 0 0 0 0 0 0...|
|21132951|0 0 0 1 0 0 0 0 0...|
+--------+--------------------+
only showing top 20 rows



In [ ]:
### WRITE TO CSV
convertToCSV = convertToCSV.withColumn('movie_id', col('movie_id').cast(IntegerType()))
convertToCSV.printSchema()
convertToCSV.write.csv('Part2.csv')

root
 |-- movie_id: integer (nullable = true)
 |-- prediction: string (nullable = true)

